<a href="https://colab.research.google.com/github/MichalMiszcz/ai_playing_music/blob/main/LSTM_autoenkoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Biblioteki

In [195]:
import torch
import torch.nn as nn
import torch.optim as optim

# Zmienne

In [194]:
seq_len=90

batch_size_var = 16
learning_rate = 0.001
weight_decay = 0.000001
epochs = 250
es_patience = 5

rnn_layers = 1
hidden_dim = 32

# Generowanie przykładowych danych

In [196]:
import random

random_notes = random.choices(range(1, 9), k=seq_len)
random_times = random.choices(range(1, 9), k=seq_len)

sequences_list = []

for n in range(2048):
  lstm_sequence = [[n, random_times[n]] for n in random_notes]
  sequences_list.append(lstm_sequence)


In [197]:
print(sequences_list)

[[[8, 4], [5, 4], [7, 2], [7, 2], [3, 1], [3, 1], [6, 3], [2, 8], [3, 1], [1, 2], [7, 2], [1, 2], [4, 5], [2, 8], [3, 1], [7, 2], [5, 4], [5, 4], [7, 2], [7, 2], [2, 8], [8, 4], [8, 4], [1, 2], [5, 4], [4, 5], [1, 2], [4, 5], [2, 8], [2, 8], [8, 4], [4, 5], [5, 4], [2, 8], [5, 4], [5, 4], [7, 2], [4, 5], [6, 3], [1, 2], [8, 4], [4, 5], [6, 3], [2, 8], [2, 8], [2, 8], [5, 4], [7, 2], [4, 5], [6, 3], [2, 8], [1, 2], [8, 4], [3, 1], [6, 3], [6, 3], [5, 4], [1, 2], [8, 4], [2, 8], [6, 3], [6, 3], [2, 8], [2, 8], [8, 4], [5, 4], [7, 2], [5, 4], [1, 2], [2, 8], [4, 5], [5, 4], [2, 8], [8, 4], [6, 3], [1, 2], [4, 5], [2, 8], [4, 5], [5, 4], [6, 3], [4, 5], [4, 5], [8, 4], [1, 2], [3, 1], [3, 1], [1, 2], [8, 4], [5, 4]], [[8, 4], [5, 4], [7, 2], [7, 2], [3, 1], [3, 1], [6, 3], [2, 8], [3, 1], [1, 2], [7, 2], [1, 2], [4, 5], [2, 8], [3, 1], [7, 2], [5, 4], [5, 4], [7, 2], [7, 2], [2, 8], [8, 4], [8, 4], [1, 2], [5, 4], [4, 5], [1, 2], [4, 5], [2, 8], [2, 8], [8, 4], [4, 5], [5, 4], [2, 8], [5, 

# Architektura sieci

In [203]:
class AutoenkoderLSTM(nn.Module):
    def __init__(self, hidden_dim=64, input_dim=2, output_dim=2, rnn_layers=2, batch_size=8, max_seq_len=90) -> None:
        super(AutoenkoderLSTM, self).__init__()
        self.max_seq_len = max_seq_len
        self.hidden_dim = hidden_dim

        self.encoder = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=rnn_layers, dropout=0.33, batch_first=True)
        self.decoder = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=rnn_layers, dropout=0.33, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_dim)
        self.encoder_linear = nn.Linear(max_seq_len * hidden_dim * batch_size, hidden_dim)
        self.proj_h = nn.Linear(hidden_dim, hidden_dim * rnn_layers)
        self.proj_c = nn.Linear(hidden_dim, hidden_dim * rnn_layers)
        self.output_dim = output_dim

        self.batch_size = batch_size

    def forward(self, x):
        batch_size = self.batch_size

        encoder_output, (h0, c0) = self.encoder(x)

        # input_seq = torch.zeros(batch_size, 1, self.output_dim).to(x.device)
        input_seq = torch.cat([torch.zeros(batch_size, 1, self.output_dim).to(x.device), x[:, :-1, :]], dim=1)

        print(self.max_seq_len * self.hidden_dim * batch_size)
        encoder_output = encoder_output.reshape(-1)
        print(encoder_output.size())
        encoder_output = self.encoder_linear(encoder_output)
        print(encoder_output.size())
        encoder_output = encoder_output.view(-1, hidden_dim)
        print(encoder_output.size())

        h0 = self.proj_h(encoder_output).view(batch_size, self.decoder.num_layers, -1).transpose(0, 1).contiguous()
        c0 = self.proj_c(encoder_output).view(batch_size, self.decoder.num_layers, -1).transpose(0, 1).contiguous()

        output, _ = self.decoder(input_seq, (h0, c0))

        output = self.linear(output)
        # output = torch.sigmoid(output)
        return output

        # self.linear.flatten_parameters()



# Dataset

In [200]:
from torch.utils.data import Dataset, DataLoader

class SequenceDataset(Dataset):
    def __init__(self, sequences):
        self.data = torch.tensor(sequences, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


# Uczenie

In [201]:
train_list = sequences_list[:int(len(sequences_list)*0.9)]
test_list = sequences_list[int(len(sequences_list)*0.9):]

print(len(train_list))
print(len(test_list))

train_dataset = SequenceDataset(train_list)
test_dataset = SequenceDataset(test_list)

train_loader = DataLoader(train_dataset, batch_size=batch_size_var, shuffle=True)
test_loader = DataLoader(test_dataset, shuffle=True)

1843
205


In [204]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = AutoenkoderLSTM(rnn_layers=rnn_layers, hidden_dim=hidden_dim, batch_size=batch_size_var, max_seq_len=seq_len).to(device)
criterion = torch.nn.MSELoss()

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

best_val_loss = float("inf")
patience_counter = 0
best_midi, best_output = None, None

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for i, midi_batch in enumerate(train_loader):
        midi_batch = midi_batch.to(device, non_blocking=True)

        optimizer.zero_grad()
        output = model(midi_batch)
        loss = criterion(output, midi_batch)
        loss.backward()

        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    model.eval()
    val_loss = 0

    sample_midi = None
    sample_output = None
    with torch.no_grad():
        for midi_batch in test_loader:
            midi_batch = midi_batch.to(device, non_blocking=True)
            outputs = model(midi_batch)

            sample_midi, sample_output = midi_batch, outputs

            val_loss += criterion(outputs, midi_batch).item()
    val_loss /= len(test_loader)

    print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.6f}")
    print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {val_loss:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # torch.save(model.state_dict(), '/content/drive/MyDrive/modele_ai/lstm_autoencoder_best.pth')
        torch.save(model.state_dict(), 'lstm_autoencoder_best.pth')
        print("Model saved as 'lstm_autoencoder_best.pth'")

        best_midi, best_output = midi.to('cpu').numpy(), outputs.to('cpu').numpy()
    else:
        patience_counter += 1

    if patience_counter >= es_patience:
        print(f"Early stopping at epoch {epoch}")

        print()
        print("Midi: ", best_midi)
        print("Outputs: ", best_output)
        print()
        break


Using device: cuda
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080
torch.Size([46080])
torch.Size([512])
torch.Size([16, 32])
46080


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 16 but got size 3 for tensor number 1 in the list.

In [60]:
# proj_h = nn.Linear(2, 16)
# x = [[0.1, 0.2]]
# x = torch.tensor(x)
# print(x)
# proj_1 = proj_h(x)
# # view_1 = x.view(2, 2, -1).transpose(0, 1).contiguous()
# proj = proj_h(x).view(2, 2, -1).transpose(0, 1).contiguous()
#
# print(proj_1)
# print(proj)

tensor([[0.1000, 0.2000]])
tensor([[-0.0599, -0.3562,  0.1286,  0.4143,  0.0901,  0.5203,  0.4646,  0.0214,
         -0.0503, -0.3505, -0.3407, -0.0952, -0.4608,  0.2438,  0.1360,  0.5337]],
       grad_fn=<AddmmBackward0>)
tensor([[[-0.0599, -0.3562,  0.1286,  0.4143],
         [-0.0503, -0.3505, -0.3407, -0.0952]],

        [[ 0.0901,  0.5203,  0.4646,  0.0214],
         [-0.4608,  0.2438,  0.1360,  0.5337]]], grad_fn=<CloneBackward0>)
